In [182]:
import pyspark
import pandas as pd
import sklearn
import findspark
import functools as ft
import pickle
import nltk
import string
import re
import tensorflow

In [183]:
from nltk.corpus import stopwords
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

from nltk.stem import WordNetLemmatizer

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [184]:
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

In [185]:
import pyspark
import findspark
from pyspark import SparkContext

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import Evaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [186]:
import os, sys
from pyspark.sql import SparkSession
import pyspark.sql.dataframe

findspark.init()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

spark = SparkSession.builder.master('local[*]').appName("ml_example").getOrCreate()

sc = spark.sparkContext

In [187]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import LinearSVCTrainingSummary

In [188]:


psy_df = pd.read_csv('data\Youtube01-Psy.csv')
katy_df = pd.read_csv('data\Youtube02-KatyPerry.csv')
lmfao_df = pd.read_csv('data\Youtube03-LMFAO.csv')
eminem_df = pd.read_csv('data\Youtube04-Eminem.csv')
shakira_df = pd.read_csv('data\Youtube05-Shakira.csv')

dataframes = [psy_df, katy_df, lmfao_df, eminem_df, shakira_df]

main_df = ft.reduce(lambda left, right: pd.concat([left, right], axis=0, ignore_index=True), dataframes)
main_df

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1
...,...,...,...,...,...
1951,_2viQ_Qnc6-bMSjqyL1NKj57ROicCSJV5SwTrw-RFFA,Katie Mettam,2013-07-13T13:27:39.441000,I love this song because we sing it at Camp al...,0
1952,_2viQ_Qnc6-pY-1yR6K2FhmC5i48-WuNx5CumlHLDAI,Sabina Pearson-Smith,2013-07-13T13:14:30.021000,I love this song for two reasons: 1.it is abou...,0
1953,_2viQ_Qnc6_k_n_Bse9zVhJP8tJReZpo8uM2uZfnzDs,jeffrey jules,2013-07-13T12:09:31.188000,wow,0
1954,_2viQ_Qnc6_yBt8UGMWyg3vh0PulTqcqyQtdE7d4Fl0,Aishlin Maciel,2013-07-13T11:17:52.308000,Shakira u are so wiredo,0


## Steps to Follow:-
1. Remove Stopwords
2. Remove emojis
3. Remove punctuation
4. Lemmatization
5. Tokenization
6. Scaling
7. Implement Spark

In [189]:
main_df['CLASS'].value_counts()

1    1005
0     951
Name: CLASS, dtype: int64

In [190]:
main_df.drop(['COMMENT_ID', 'AUTHOR', 'DATE'], axis=1, inplace=True)
main_df

,CONTENT,CLASS
0,"Huh, anyway check out this you[tube] channel: ...",1
1,Hey guys check out my new channel and our firs...,1
2,just for test I have to say murdev.com,1
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,watch?v=vtaRGgvGtWQ Check this out .﻿,1
...,...,...
1951,I love this song because we sing it at Camp al...,0
1952,I love this song for two reasons: 1.it is abou...,0
1953,wow,0
1954,Shakira u are so wiredo,0


## Adding single letters to the list of stopwords

In [191]:
alphabet_list = list(string.ascii_lowercase)
stop_words = stopwords.words('english')

for i in range(len(alphabet_list)):
    if (alphabet_list[i] in stop_words):
        continue
    else:
        stop_words.append(alphabet_list[i])
        
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Removing Punctuations and Random Characters

In [192]:
def clean_text(df, text):
    df[text] = df[text].str.lower()
    df[text] = df[text].apply(lambda elem: re.sub(r'@[A-Za-z0-9]+', '', elem))              # Removing all @mentions
    df[text] = df[text].apply(lambda elem: re.sub(r'https?:\/\/\S+', '', elem))             # Removing all links
    df[text] = df[text].apply(lambda elem: re.sub(r'#', '', elem))                          # Removing the hashtag symbol
    df[text] = df[text].apply(lambda elem: re.sub(r"([^0-9A-Za-z \t])", "", elem))          # Removing all non-alphanumeric symbols
    df[text] = df[text].apply(lambda elem: re.sub(r"\w*\d\w*", "", elem))                   # Removing all numbers
    df[text] = df[text].apply(lambda elem: re.sub(r"^\d+\w|^\w+\d+\w|^\w+\d$", "", elem))
    
    return df

In [193]:
clean_df = clean_text(main_df, 'CONTENT')
clean_df

,CONTENT,CLASS
0,huh anyway check out this youtube channel,1
1,hey guys check out my new channel and our firs...,1
2,just for test i have to say murdevcom,1
3,me shaking my sexy ass on my channel enjoy,1
4,watchvvtarggvgtwq check this out,1
...,...,...
1951,i love this song because we sing it at camp al...,0
1952,i love this song for two reasons is about afr...,0
1953,wow,0
1954,shakira u are so wiredo,0


## Removing Stopwords

In [194]:
def remove_stopwords(df, text):
    ABC = list((df[text]))
    clean_lst = []
    for i in range(ABC.__len__()):
        result_words = [word for word in ABC[i].split(' ') if word not in stop_words]
        str_result = ' '.join(result_words)
        clean_lst.append(str_result)
        
    cleaner_df = pd.DataFrame(clean_lst, columns=['CONTENT'])
    
    return cleaner_df

cleaner_df = remove_stopwords(clean_df, 'CONTENT')
cleaner_df.head(60)

,CONTENT
0,huh anyway check youtube channel
1,hey guys check new channel first vid us monke...
2,test say murdevcom
3,shaking sexy ass channel enjoy
4,watchvvtarggvgtwq check
5,hey check new website site kids stuff kidsmedi...
6,subscribe channel
7,turned mute soon came wanted check views
8,check channel funny videos
9,shouldd check channel tell next


## Removing Emojis

In [195]:
cleaner_df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
# cleaner_df.to_csv('data/temp_data.csv',index=False)

,CONTENT
0,huh anyway check youtube channel
1,hey guys check new channel first vid us monke...
2,test say murdevcom
3,shaking sexy ass channel enjoy
4,watchvvtarggvgtwq check
...,...
1951,love song sing camp time
1952,love song two reasons africa born beautiful ...
1953,wow
1954,shakira wiredo


## Lemmatization

In [196]:
lemmatizer = WordNetLemmatizer()

def get_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'V': wordnet.VERB,
        'N': wordnet.NOUN,
        'J': wordnet.ADJ,
        'R': wordnet.ADV
    }
    
    return tag_dict.get(tag)

def lem_df(df, text):
    lst_sent = list(df[text])
    lem_lst = []
    for i in range(len(lst_sent)):
        input = nltk.word_tokenize(lst_sent[i])
        result_words = [lemmatizer.lemmatize(word) for word in input]
        str_result = ' '.join(result_words)
        lem_lst.append(str_result)
        
    cleanest_df = pd.DataFrame(lem_lst, columns=['CONTENT'])
    return cleanest_df

lemmatized_df = lem_df(cleaner_df, 'CONTENT')
lemmatized_df

,CONTENT
0,huh anyway check youtube channel
1,hey guy check new channel first vid u monkey i...
2,test say murdevcom
3,shaking sexy as channel enjoy
4,watchvvtarggvgtwq check
...,...
1951,love song sing camp time
1952,love song two reason africa born beautiful sou...
1953,wow
1954,shakira wiredo


## Tokenization

In [197]:
def tokenize_df(df, col_name):
    sentences = []
    
    length = df[col_name].__len__()
    
    for i in range(length):
        sentences.append(df[col_name][i])
        
    tokenizer = Tokenizer(num_words=3400, oov_token='<00V>')
    tokenizer.fit_on_texts(sentences)
    word_index = tokenizer.word_index
    # print(word_index)
    
    # Sequencing
    
    sequences = tokenizer.texts_to_sequences(sentences)
    sequences = pad_sequences(sequences, padding='post', maxlen=16)
    
    return sequences


    
tokenized_seq = tokenize_df(lemmatized_df, 'CONTENT')

In [198]:
tokenized_train = pd.DataFrame(tokenized_seq)
tokenized_train = tokenized_train.join(main_df['CLASS'])
tokenized_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,CLASS
0,633,835,2,8,10,0,0,0,0,0,0,0,0,0,0,0,1
1,2,14,10,82,433,40,634,18,634,376,1306,169,5,17,6,7,1
2,1307,208,1308,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1309,209,377,10,210,0,0,0,0,0,0,0,0,0,0,0,1
4,1310,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1951,9,4,290,1,29,0,0,0,0,0,0,0,0,0,0,0,0
1952,9,4,564,269,337,757,105,969,337,0,0,0,0,0,0,0,0
1953,123,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1954,55,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Upsampling

In [199]:
tokenized_train['CLASS'].value_counts()

train_majority = tokenized_train[tokenized_train.CLASS == 1]
train_minority = tokenized_train[tokenized_train.CLASS == 0]

minority_upsampled = resample(train_minority, replace=True, n_samples=train_majority.__len__(), random_state=123)
minority_upsampled['CLASS'].value_counts()

train_upsampled = pd.concat([train_majority, minority_upsampled], ignore_index=True)
print(train_upsampled['CLASS'].value_counts())
train_upsampled

1    1005
0    1005
Name: CLASS, dtype: int64


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,CLASS
0,633,835,2,8,10,0,0,0,0,0,0,0,0,0,0,0,1
1,2,14,10,82,433,40,634,18,634,376,1306,169,5,17,6,7,1
2,1307,208,1308,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1309,209,377,10,210,0,0,0,0,0,0,0,0,0,0,0,1
4,1310,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005,24,41,160,417,0,0,0,0,0,0,0,0,0,0,0,0,0
2006,284,131,113,317,131,171,157,130,343,0,0,0,0,0,0,0,0
2007,125,244,673,39,155,104,83,0,0,0,0,0,0,0,0,0,0
2008,658,171,157,130,343,171,157,130,343,0,0,0,0,0,0,0,0


## Scaling

In [200]:
""" 
scaler = StandardScaler()
scaler.fit(train_upsampled.drop('CLASS', axis=1))
scaled_feature = scaler.transform(train_upsampled.drop('CLASS', axis=1))
scaled_df = pd.DataFrame(scaled_feature, columns = train_upsampled.columns[:-1])
# scaled_df = pd.concat([scaled_df, train_upsampled['CLASS']], axis=1, ignore_index=True)
scaled_df  = scaled_df.join(train_upsampled['CLASS'])
scaled_df """

" \nscaler = StandardScaler()\nscaler.fit(train_upsampled.drop('CLASS', axis=1))\nscaled_feature = scaler.transform(train_upsampled.drop('CLASS', axis=1))\nscaled_df = pd.DataFrame(scaled_feature, columns = train_upsampled.columns[:-1])\n# scaled_df = pd.concat([scaled_df, train_upsampled['CLASS']], axis=1, ignore_index=True)\nscaled_df  = scaled_df.join(train_upsampled['CLASS'])\nscaled_df "

In [201]:
""" final_train = pd.DataFrame(X_train).join(y_train)
final_test = pd.DataFrame(X_test).join(y_test)
print(final_train['CLASS'].value_counts())
print(final_test['CLASS'].value_counts()) """

" final_train = pd.DataFrame(X_train).join(y_train)\nfinal_test = pd.DataFrame(X_test).join(y_test)\nprint(final_train['CLASS'].value_counts())\nprint(final_test['CLASS'].value_counts()) "

## Convert to CSV

In [202]:
train_upsampled.to_csv('data/clean_data.csv', index=False)

## Implementing Spark

In [203]:
training_data = spark.read.csv('data/clean_data.csv', header=True, inferSchema=True)
training_data.printSchema()


root
 |-- 0: integer (nullable = true)
 |-- 1: integer (nullable = true)
 |-- 2: integer (nullable = true)
 |-- 3: integer (nullable = true)
 |-- 4: integer (nullable = true)
 |-- 5: integer (nullable = true)
 |-- 6: integer (nullable = true)
 |-- 7: integer (nullable = true)
 |-- 8: integer (nullable = true)
 |-- 9: integer (nullable = true)
 |-- 10: integer (nullable = true)
 |-- 11: integer (nullable = true)
 |-- 12: integer (nullable = true)
 |-- 13: integer (nullable = true)
 |-- 14: integer (nullable = true)
 |-- 15: integer (nullable = true)
 |-- CLASS: integer (nullable = true)



In [204]:
## Creating a Features Array
# The VectorAssembler makes a single vector from all the feature columns to be used as feature input

feature_cols = training_data.columns[:-1]

vect_assmebler = VectorAssembler(inputCols=feature_cols, outputCol='Features')


In [205]:
data_w_features = vect_assmebler.transform(training_data)
data_w_features.show()

+----+----+----+---+----+----+---+----+----+----+----+---+---+---+---+----+-----+--------------------+
|   0|   1|   2|  3|   4|   5|  6|   7|   8|   9|  10| 11| 12| 13| 14|  15|CLASS|            Features|
+----+----+----+---+----+----+---+----+----+----+----+---+---+---+---+----+-----+--------------------+
| 633| 835|   2|  8|  10|   0|  0|   0|   0|   0|   0|  0|  0|  0|  0|   0|    1|(16,[0,1,2,3,4],[...|
|   2|  14|  10| 82| 433|  40|634|  18| 634| 376|1306|169|  5| 17|  6|   7|    1|[2.0,14.0,10.0,82...|
|1307| 208|1308|  0|   0|   0|  0|   0|   0|   0|   0|  0|  0|  0|  0|   0|    1|(16,[0,1,2],[1307...|
|1309| 209| 377| 10| 210|   0|  0|   0|   0|   0|   0|  0|  0|  0|  0|   0|    1|(16,[0,1,2,3,4],[...|
|1310|   2|   0|  0|   0|   0|  0|   0|   0|   0|   0|  0|  0|  0|  0|   0|    1|(16,[0,1],[1310.0...|
|  22|   2|  14| 72| 170| 268|378|1311| 236|   0|   0|  0|  0|  0|  0|   0|    1|(16,[0,1,2,3,4,5,...|
|   7|  10|   0|  0|   0|   0|  0|   0|   0|   0|   0|  0|  0|  0|  0|   

In [206]:
# Creating Final Dataframe to feed into the model

final_data = data_w_features.select('Features', 'CLASS')
final_data.show()

+--------------------+-----+
|            Features|CLASS|
+--------------------+-----+
|(16,[0,1,2,3,4],[...|    1|
|[2.0,14.0,10.0,82...|    1|
|(16,[0,1,2],[1307...|    1|
|(16,[0,1,2,3,4],[...|    1|
|(16,[0,1],[1310.0...|    1|
|(16,[0,1,2,3,4,5,...|    1|
|(16,[0,1],[7.0,10...|    1|
|(16,[0,1,2,3],[2....|    1|
|(16,[0,1,2,3,4],[...|    1|
|(16,[0,1],[22.0,7...|    1|
|[118.0,146.0,177....|    1|
|          (16,[],[])|    1|
|(16,[0,1,2],[7.0,...|    1|
|(16,[0,1],[6.0,5.0])|    1|
|[6.0,2.0,10.0,7.0...|    1|
|          (16,[],[])|    1|
|          (16,[],[])|    1|
|[212.0,1326.0,212...|    1|
|(16,[0,1,2],[7.0,...|    1|
|(16,[0,1,2],[437....|    1|
+--------------------+-----+
only showing top 20 rows



### Splitting training and testing data

In [207]:
train_data, test_data = final_data.randomSplit([0.7,0.3])


In [208]:
lsvc = LinearSVC(featuresCol='Features', labelCol='CLASS')

columns = ['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', '_c23', '_c24', '_c25']

# Fitting the model
lsvc_model = lsvc.fit(train_data)

pred = lsvc_model.transform(test_data)
predDF = pred.toPandas()
predDF.head()

,Features,CLASS,rawPrediction,prediction
0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.999989144174332, -0.999989144174332]",0.0
1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.999989144174332, -0.999989144174332]",0.0
2,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.999989144174332, -0.999989144174332]",0.0
3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.999989144174332, -0.999989144174332]",0.0
4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.999989144174332, -0.999989144174332]",0.0


In [212]:

evaluator=MulticlassClassificationEvaluator(metricName="accuracy", labelCol='CLASS', predictionCol='prediction')
acc = evaluator.evaluate(pred)
print("Prediction Accuracy: ", acc)


Prediction Accuracy:  0.5704584040747029


In [210]:
""" lsvc_sum = LinearSVCTrainingSummary()
print(lsvc_sum.accuracy(lsvc_model)) """

' lsvc_sum = LinearSVCTrainingSummary()\nprint(lsvc_sum.accuracy(lsvc_model)) '